In [2]:
import scanpy as sc
import matplotlib as plt
import anndata

## 1. Spatial data:
##### sc.pp.highly_variable_genes(st_data, n_top_genes = 64, layer = 'log')
##### Also, make sure the data is not logged (in the .X), since ENVI expected unlogged counts

In [3]:
#withhelded = sc.read_h5ad('/work/rwth1209/dana_projects/test_new_imputation_tools/enVI/ref/withheld_spatial_vk12_coarse_annotation.h5ad')
complete = sc.read_h5ad('/work/rwth1209/dana_projects/test_new_imputation_tools/enVI/ref/complete_spatial_vk12_coarse_annotation.h5ad')
scref = sc.read_h5ad("/work/rwth1209/dana_projects/test_new_imputation_tools/enVI/ref/snRNA_teichman_atlas_normalized.h5ad")
scref_raw = sc.read_h5ad("/work/rwth1209/dana_projects/test_new_imputation_tools/enVI/ref/snRNA_teichman_atlas_raw.h5ad")

/work/rwth1209/enviroments/envi/lib/python3.9/site-packages/anndata/_core/anndata.py:1820: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


In [4]:
scref.X.max()

9.030837

In [5]:
scref.X

<257029x32732 sparse matrix of type '<class 'numpy.float32'>'
	with 372161202 stored elements in Compressed Sparse Row format>

In [6]:
# sc.pp.normalize_total(scref)
# sc.pp.log1p(scref)
# sc.pp.pca(scref, n_comps=30)

In [7]:
sc.pp.highly_variable_genes(scref, n_top_genes = 10000)

/work/rwth1209/enviroments/envi/lib/python3.9/site-packages/scanpy/preprocessing/_highly_variable_genes.py:226: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  disp_grouped = df.groupby("mean_bin")["dispersions"]


In [8]:
scref.X.max()

9.030837

In [9]:
scref_raw.X.max()

17441.0

In [10]:
scref.X = scref_raw.X

In [11]:
scref.X.max()

17441.0

In [12]:
#filter out genes; only keep hvg and which are in st data
import numpy as np

highly_variable_genes = scref.var['highly_variable']

genes_to_keep = list(complete.var_names)
genes_to_keep_set = set(genes_to_keep)

combined_genes = np.union1d(np.where(highly_variable_genes)[0], np.where(scref.var_names.isin(genes_to_keep_set))[0])
scref = scref[:, combined_genes]

In [13]:
scref

View of AnnData object with n_obs × n_vars = 257029 × 10102
    obs: 'sangerID', 'donor', 'donor_type', 'region', 'age', 'gender', 'facility', 'cell_or_nuclei', 'modality', 'kit_10x', 'flushed', 'cell_type', 'cell_state', 'n_genes', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'total_counts_ribo', 'pct_counts_ribo', 'scrublet_score'
    var: 'gene_name_scRNA-0-original', 'gene_name_snRNA-1-original', 'gene_name_multiome-2-original', 'gene_id', 'highly_variable', 'means', 'dispersions', 'dispersions_norm'
    uns: 'age_colors', 'cell_or_nuclei_colors', 'cell_state_colors', 'cell_type_colors', 'donor_colors', 'donor_type_colors', 'facility_colors', 'flushed_colors', 'gender_colors', 'kit_10x_colors', 'leiden', 'log1p', 'modality_colors', 'neighbors', 'original_or_new_colors', 'region_colors', 'region_finest_colors', 'scANVI_predictions_colors', 'umap', 'hvg'
    obsm: 'X_umap'
    obsp: 'connectivities', 'distances'

In [14]:
scref.X.max()

17441.0

In [15]:
scref.X.shape

(257029, 10102)

In [16]:
array_X = (scref.X).toarray() 

In [17]:
array_X

array([[0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [19]:
del scref.X

In [20]:
scref.X = array_X

In [23]:
scref.X.max()

17441.0

In [30]:
del scref.var['gene_name_scRNA-0-original']

In [31]:
scref.var

,gene_id,highly_variable,means,dispersions,dispersions_norm
gene_name-new,,,,,
AL732372.1,ENSG00000236601,True,0.000097,2.299262,1.074797
LINC01409,ENSG00000237491,True,0.317450,2.042045,0.480314
AL645608.6,ENSG00000272438,True,0.003774,2.027749,0.447274
LINC02593,ENSG00000223764,True,0.006575,1.999187,0.381259
SAMD11,ENSG00000187634,True,0.031996,1.963435,0.298630
...,...,...,...,...,...
MT-ND4,ENSG00000198886,True,3.155295,4.713407,1.170054
MT-CYB,ENSG00000198727,True,3.049900,4.536783,0.695219
AC136616.3,ENSG00000278782,True,0.000476,1.983164,0.344228


In [32]:
scref.write('ref_10000_array.h5ad')

In [100]:
# remove 3 genes that are not in scRNA ref data
shared = scref.var_names.intersection(withhelded.var_names)
withhelded = withhelded[:, shared]

In [101]:
len(shared)

492

In [102]:
withhelded

View of AnnData object with n_obs × n_vars = 217184 × 492
    obs: 'fov', 'volume', 'min_x', 'min_y', 'max_x', 'max_y', 'anisotropy', 'transcript_count', 'perimeter_area_ratio', 'solidity', 'Cellbound2_raw', 'Cellbound2_high_pass', 'Cellbound3_raw', 'Cellbound3_high_pass', 'PolyT_raw', 'PolyT_high_pass', 'Cellbound1_raw', 'Cellbound1_high_pass', 'DAPI_raw', 'DAPI_high_pass', 'region', 'n_counts', 'leiden_0_5', 'leiden_1', 'leiden_1_5', 'tacco', 'cell_type'
    var: 'mean', 'std'
    uns: 'leiden', 'leiden_1_5_colors', 'leiden_1_colors', 'log1p', 'neighbors', 'pca', 'rank_genes_groups', 'spatial', 'tacco_colors', 'umap'
    obsm: 'X_pca', 'X_umap', 'blank_genes', 'spatial', 'tacco'
    varm: 'PCs', 'tacco'
    layers: 'counts'
    obsp: 'connectivities', 'distances'

In [103]:
#sc.pp.highly_variable_genes(withhelded, n_top_genes = 64)

In [104]:
withhelded.var

,mean,std
ISG15,0.004497,0.062911
KAZN,0.008519,0.067898
IGSF21,0.002604,0.040328
PLA2G2A,0.011298,0.111351
C1QA,0.081590,0.275445
...,...,...
SREBF2,0.016741,0.105352
FBLN1,0.022687,0.132274
APP,0.140145,0.316129
MX2,0.005374,0.061484


In [105]:
withhelded.X

ArrayView([[-0.07147601, -0.12546673, -0.06456725, ..., -0.44331616,
            -0.08741081, -0.0529125 ],
           [-0.07147601, -0.12546673, -0.06456725, ...,  1.6901658 ,
            -0.08741081, -0.0529125 ],
           [-0.07147601, -0.12546673, -0.06456725, ..., -0.44331616,
            -0.08741081, -0.0529125 ],
           ...,
           [-0.07147601, -0.12546673, -0.06456725, ..., -0.44331616,
            -0.08741081, -0.0529125 ],
           [-0.07147601, -0.12546673, -0.06456725, ..., -0.44331616,
            -0.08741081, -0.0529125 ],
           [-0.07147601, -0.12546673, -0.06456725, ..., -0.44331616,
            -0.08741081, -0.0529125 ]], dtype=float32)

In [106]:
#Method 1 Compressed Sparse Row matrix
#replace normalized withhelded.X with raw counts (withhelded.X was anarray so I had to convert it to matrix)

import numpy as np
from scipy.sparse import csr_matrix

matrix_X = csr_matrix(withhelded.X)

In [107]:
del withhelded.X

In [108]:
withhelded.X = matrix_X

In [109]:
withhelded.X.shape

(217184, 492)

In [110]:
withhelded.X

<217184x492 sparse matrix of type '<class 'numpy.float32'>'
	with 106854528 stored elements in Compressed Sparse Row format>

In [111]:
withhelded.layers['counts'].shape

(217184, 492)

In [112]:
withhelded.X = withhelded.layers['counts']

In [113]:
withhelded.layers['counts'].max()

421.0

In [114]:
withhelded.X.max()

421.0

In [115]:
withhelded.X = (withhelded.X).toarray() 

In [122]:
withhelded.X

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [124]:
withhelded

AnnData object with n_obs × n_vars = 217184 × 492
    obs: 'fov', 'volume', 'min_x', 'min_y', 'max_x', 'max_y', 'anisotropy', 'transcript_count', 'perimeter_area_ratio', 'solidity', 'Cellbound2_raw', 'Cellbound2_high_pass', 'Cellbound3_raw', 'Cellbound3_high_pass', 'PolyT_raw', 'PolyT_high_pass', 'Cellbound1_raw', 'Cellbound1_high_pass', 'DAPI_raw', 'DAPI_high_pass', 'region', 'n_counts', 'leiden_0_5', 'leiden_1', 'leiden_1_5', 'tacco', 'cell_type'
    var: 'mean', 'std'
    uns: 'leiden', 'leiden_1_5_colors', 'leiden_1_colors', 'log1p', 'neighbors', 'pca', 'rank_genes_groups', 'spatial', 'tacco_colors', 'umap'
    obsm: 'X_pca', 'X_umap', 'blank_genes', 'spatial', 'tacco'
    varm: 'PCs', 'tacco'
    layers: 'counts'
    obsp: 'connectivities', 'distances'

In [117]:
withhelded.write('withhelded_array.h5ad')
#withhelded.write('withhelded492_raw.h5ad')

In [80]:
st_data = withhelded
sc_data  = scref

In [81]:
sc_data.X

<257029x5126 sparse matrix of type '<class 'numpy.float32'>'
	with 76941991 stored elements in Compressed Sparse Row format>

In [82]:
X_array = (sc_data.X).toarray() 

In [83]:
# apperantely andata.X is suppose to be an array and not a matrix 
sc_data.X = X_array

In [84]:
X_array.shape

(257029, 5126)

In [85]:
sc_data.layers['log'] = X_array

/tmp/ipykernel_37868/546456143.py:1: ImplicitModificationWarning: Setting element `.layers['log']` of view, initializing view as actual.
  sc_data.layers['log'] = X_array


In [86]:
sc_data

AnnData object with n_obs × n_vars = 257029 × 5126
    obs: 'sangerID', 'combinedID', 'donor', 'donor_type', 'region', 'region_finest', 'age', 'gender', 'facility', 'cell_or_nuclei', 'modality', 'kit_10x', 'flushed', 'n_genes', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'total_counts_ribo', 'pct_counts_ribo', 'scrublet_score', 'scrublet_leiden', 'cluster_scrublet_score', 'doublet_pval', 'doublet_bh_pval', 'batch_key', 'leiden_scVI', 'cell_type', 'cell_state_HCAv1', 'cell_state_scNym', 'cell_state_scNym_confidence', 'cell_state', 'latent_RT_efficiency', 'latent_cell_probability', 'latent_scale', 'n_counts', '_scvi_batch', '_scvi_labels', 'clus20', 'doublet_cls', 'original_or_new', 'batch', 'scANVI_predictions', 'leiden_scArches'
    var: 'gene_name-new', 'gene_name_scRNA-0-original', 'gene_name_snRNA-1-original', 'gene_name_multiome-2-original', 'highly_variable', 'means', 'dispersions', 'dispersions_norm'
    uns: 'age_colors', 'cell_or_nuclei_colors', 'ce

In [54]:
import scenvi 

envi_model = scenvi.ENVI(spatial_data = st_data, sc_data = sc_data)

Computing Niche Covariance Matrices


In [96]:
del sc_data.var['gene_name_multiome-2-original']

In [97]:
sc_data.var

,highly_variable,means,dispersions,dispersions_norm
gene_name-new,,,,
AL732372.1,True,0.000022,0.805383,0.982197
HES4,True,0.415288,1.332394,0.848631
ISG15,True,0.189830,1.332300,2.185377
TNFRSF18,True,0.006388,0.942452,1.295186
TNFRSF4,True,0.041176,0.930257,1.267340
...,...,...,...,...
MT-ND4L,True,0.210800,0.877994,1.148001
MT-ND4,True,1.798778,3.219527,1.336889
MT-CYB,True,1.707548,3.042903,0.799802


In [98]:
sc_data.write('ref_5000_array.h5ad')

In [126]:
st_data = sc.read_h5ad('/work/rwth1209/dana_projects/test_new_imputation_tools/enVI/data_preprocessing/withhelded_hvg64_array.h5ad')
sc_data = sc.read_h5ad("/work/rwth1209/dana_projects/test_new_imputation_tools/enVI/data_preprocessing/ref_5000_array.h5ad")

In [127]:
st_data

AnnData object with n_obs × n_vars = 217184 × 492
    obs: 'fov', 'volume', 'min_x', 'min_y', 'max_x', 'max_y', 'anisotropy', 'transcript_count', 'perimeter_area_ratio', 'solidity', 'Cellbound2_raw', 'Cellbound2_high_pass', 'Cellbound3_raw', 'Cellbound3_high_pass', 'PolyT_raw', 'PolyT_high_pass', 'Cellbound1_raw', 'Cellbound1_high_pass', 'DAPI_raw', 'DAPI_high_pass', 'region', 'n_counts', 'leiden_0_5', 'leiden_1', 'leiden_1_5', 'tacco', 'cell_type'
    var: 'mean', 'std', 'highly_variable', 'means', 'dispersions', 'dispersions_norm'
    uns: 'hvg', 'leiden', 'leiden_1_5_colors', 'leiden_1_colors', 'log1p', 'neighbors', 'pca', 'rank_genes_groups', 'spatial', 'tacco_colors', 'umap'
    obsm: 'X_pca', 'X_umap', 'blank_genes', 'spatial', 'tacco'
    varm: 'PCs', 'tacco'
    layers: 'counts'
    obsp: 'connectivities', 'distances'

In [128]:
sc_data

AnnData object with n_obs × n_vars = 257029 × 5126
    obs: 'sangerID', 'combinedID', 'donor', 'donor_type', 'region', 'region_finest', 'age', 'gender', 'facility', 'cell_or_nuclei', 'modality', 'kit_10x', 'flushed', 'n_genes', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'total_counts_ribo', 'pct_counts_ribo', 'scrublet_score', 'scrublet_leiden', 'cluster_scrublet_score', 'doublet_pval', 'doublet_bh_pval', 'batch_key', 'leiden_scVI', 'cell_type', 'cell_state_HCAv1', 'cell_state_scNym', 'cell_state_scNym_confidence', 'cell_state', 'latent_RT_efficiency', 'latent_cell_probability', 'latent_scale', 'n_counts', '_scvi_batch', '_scvi_labels', 'clus20', 'doublet_cls', 'original_or_new', 'batch', 'scANVI_predictions', 'leiden_scArches'
    var: 'highly_variable', 'means', 'dispersions', 'dispersions_norm'
    uns: 'age_colors', 'cell_or_nuclei_colors', 'cell_state_colors', 'cell_type_colors', 'donor_colors', 'donor_type_colors', 'facility_colors', 'flushed_colors'

In [129]:
st_data.X

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [133]:
sc_data.X = (sc_data.X).toarray()

In [134]:
sc_data.X

array([[  0.,   1.,   1., ..., 171.,   0.,   0.],
       [  0.,   0.,   4., ...,  98.,   0.,   0.],
       [  0.,   0.,   2., ...,  32.,   0.,   0.],
       ...,
       [  0.,   0.,   0., ...,   0.,   0.,   0.],
       [  0.,   0.,   0., ...,   0.,   0.,   0.],
       [  0.,   0.,   0., ...,   0.,   0.,   0.]], dtype=float32)

In [135]:
sc_data.write('ref_5000_array.h5ad')